In [1]:
import pennylane as qml
from pennylane import numpy as np

# Define the molecule
a = 1.5
symbols  = ['H', 'H', 'H', 'H']
geometry = np.array([[0.0,  0.0,  0.0],
                     [a,  0.0,  0.0],
                     [2*a, 0.0,  0.0],
                     [3*a, 0.0, 0.0]], requires_grad = False)
electrons = 4
charge = 0

# Build the electronic Hamiltonian
H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, charge=charge)

# Define the HF state
hf_state = qml.qchem.hf_state(electrons, qubits)

# Generate single and double excitations
singles, doubles = qml.qchem.excitations(electrons, qubits)

# Map excitations to the wires the UCCSD circuit will act on
s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)

# Define the device
dev = qml.device("default.qubit", wires=qubits)

# Define the qnode
@qml.qnode(dev)
def circuit(params, wires, s_wires, d_wires, hf_state):
    qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
    return qml.expval(H)

# Define the initial values of the circuit parameters
params = np.zeros(len(singles) + len(doubles))

# Define the optimizer
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)

# Optimize the circuit parameters and compute the energy
for n in range(20):
    params, energy = optimizer.step_and_cost(circuit, params,
    wires=range(qubits), s_wires=s_wires, d_wires=d_wires, hf_state=hf_state)
    if n % 10 == 0:
        print("step = {:},  E = {:.16f} Ha".format(n, energy))
print("step = {:},  E = {:.16f} Ha".format(n, energy))

step = 0,  E = -2.1198583667536908 Ha
step = 10,  E = -2.1653652731022812 Ha
step = 19,  E = -2.1654535966911261 Ha


In [2]:
drawer = qml.draw(circuit)
print(drawer(params, wires=range(qubits), s_wires=s_wires, d_wires=d_wires, hf_state=hf_state))

0: ─╭UCCSD(M0)─┤ ╭<𝓗>
1: ─├UCCSD(M0)─┤ ├<𝓗>
2: ─├UCCSD(M0)─┤ ├<𝓗>
3: ─├UCCSD(M0)─┤ ├<𝓗>
4: ─├UCCSD(M0)─┤ ├<𝓗>
5: ─├UCCSD(M0)─┤ ├<𝓗>
6: ─├UCCSD(M0)─┤ ├<𝓗>
7: ─╰UCCSD(M0)─┤ ╰<𝓗>

M0 = 
[ 1.15962028e-02 -8.55398143e-17  1.15962043e-02  1.64557728e-16
 -1.88601560e-17 -7.51601469e-03  8.17921446e-17 -7.51597456e-03
  1.03764849e-01  1.24480882e-17 -1.94826597e-18  6.59250097e-02
  4.77122388e-02  1.84627912e-17  1.02612572e-01 -5.49086247e-02
 -7.38663991e-18  9.47135911e-18 -5.49086112e-02  1.02612401e-01
  1.52003616e-18  4.77117339e-02  2.76576961e-01  8.81199784e-18
 -9.99150163e-18  5.94360242e-02]


In [3]:
qml.UCCSD(params, wires=range(qubits), s_wires=s_wires, d_wires=d_wires, init_state=hf_state)

UCCSD(tensor([ 1.15962028e-02, -8.55398143e-17,  1.15962043e-02,
         1.64557728e-16, -1.88601560e-17, -7.51601469e-03,
         8.17921446e-17, -7.51597456e-03,  1.03764849e-01,
         1.24480882e-17, -1.94826597e-18,  6.59250097e-02,
         4.77122388e-02,  1.84627912e-17,  1.02612572e-01,
        -5.49086247e-02, -7.38663991e-18,  9.47135911e-18,
        -5.49086112e-02,  1.02612401e-01,  1.52003616e-18,
         4.77117339e-02,  2.76576961e-01,  8.81199784e-18,
        -9.99150163e-18,  5.94360242e-02], requires_grad=True), wires=[0, 1, 2, 3, 4, 5, 6, 7])

In [4]:
singles, doubles = qml.qchem.excitations(electrons, qubits)
singles

[[0, 4], [0, 6], [1, 5], [1, 7], [2, 4], [2, 6], [3, 5], [3, 7]]

In [5]:
doubles

[[0, 1, 4, 5],
 [0, 1, 4, 7],
 [0, 1, 5, 6],
 [0, 1, 6, 7],
 [0, 2, 4, 6],
 [0, 3, 4, 5],
 [0, 3, 4, 7],
 [0, 3, 5, 6],
 [0, 3, 6, 7],
 [1, 2, 4, 5],
 [1, 2, 4, 7],
 [1, 2, 5, 6],
 [1, 2, 6, 7],
 [1, 3, 5, 7],
 [2, 3, 4, 5],
 [2, 3, 4, 7],
 [2, 3, 5, 6],
 [2, 3, 6, 7]]

In [6]:
len(doubles)

18

In [7]:
hf_state

array([1, 1, 1, 1, 0, 0, 0, 0])

In [9]:
type(H)

pennylane.ops.op_math.sum.Sum